In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.__version__

In [0]:
#cifar10 데이터셋을 다운로드 받기 위해 keras load_data함수를 import
from tensorflow.keras.datasets.cifar10 import load_data

#CIFAR10 데이터셋을 다운로드하고 불러온다.
(x_train, y_train),(x_test, y_test) = load_data()

# train데이터의 이미지 배열과 레이블의 shape을 확인
print(x_train.shape)
print(y_train.shape)

#test데이터의 이미지 배열과 레이블의 shape을 확인
print(x_test.shape)
print(y_test.shape)

In [0]:
plt.figure(figsize=(3, 3))
plt.imshow(x_train[4])
plt.show()

In [0]:
#다음 배치를 읽어오기 위한 next_batch함수를 정의한다.
# num갯수 만큼 랜덤한 샘플이미지들과 레이블들을 리턴한다.
def next_batch(num, data, labels):
  idx = np.arange(0, len(data))
  np.random.shuffle(idx)
  idx=idx[:num]
  data_shuffle=[data[i] for i in idx]
  labels_shuffle=[labels[i] for i in idx]

  return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [0]:
#CNN모델을 정의한다.
def build_CNN_classifier(x):
  x_image = x

  #1st conv2d
  # 5x5 64 filter(channel)
  # 32x32x3 -> 32x32x64
  W_conv1 = tf.Variable(tf.truncated_normal(shape=[5,5,3,64], stddev=5e-2))
  b_conv1 = tf.Variable(tf.constant(0.1, shape=[64]))
  h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1,strides=[1,1,1,1],padding='SAME')+b_conv1)

  #1st pooling Layer
  # 32x32x64 -> 16x16x64
  h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')

  # 2nd conv2d
  # 5x5 64 filter(channel)
  # 16x16x64 -> 16x16x64
  W_conv2 = tf.Variable(tf.truncated_normal(shape=[5,5,64,64], stddev=5e-2))
  b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
  h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2,strides=[1,1,1,1],padding='SAME')+b_conv2)

  # 2nd pooling Layer
  # 16x16x64 -> 8x8x64
  h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')

  # 3rd conv2d
  # 3x3 128 filter(channel)
  # 8x8x64 -> 8x8x128
  W_conv3 = tf.Variable(tf.truncated_normal(shape=[3,3,64,128], stddev=5e-2))
  b_conv3 = tf.Variable(tf.constant(0.1, shape=[128]))
  h_conv3 = tf.nn.relu(tf.nn.conv2d(h_pool2, W_conv3,strides=[1,1,1,1],padding='SAME')+b_conv3)

  # 4th conv2d
  # 3x3 128 filter(channel)
  # 8x8x128 -> 8x8x128
  W_conv4 = tf.Variable(tf.truncated_normal(shape=[3,3,128,128], stddev=5e-2))
  b_conv4 = tf.Variable(tf.constant(0.1, shape=[128]))
  h_conv4 = tf.nn.relu(tf.nn.conv2d(h_conv3, W_conv4,strides=[1,1,1,1],padding='SAME')+b_conv4)

  # 5th conv2d
  # 3x3 128 filter(channel)
  # 8x8x128 -> 8x8x128
  W_conv5 = tf.Variable(tf.truncated_normal(shape=[3,3,128,128], stddev=5e-2))
  b_conv5 = tf.Variable(tf.constant(0.1, shape=[128]))
  h_conv5 = tf.nn.relu(tf.nn.conv2d(h_conv4, W_conv5,strides=[1,1,1,1],padding='SAME')+b_conv5)

  # Fully Connected Layer
  # 8x8x128(8192) -> 384 (Features)
  W_fc1 = tf.Variable(tf.truncated_normal(shape=[8*8*128, 384], stddev=5e-2))
  b_fc1 = tf.Variable(tf.constant(0.1, shape=[384]))

  h_conv5_flat = tf.reshape(h_conv5, [-1, 8*8*128])
  h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1)+b_fc1)

  # dropout을 구현
  h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

  # 384 -> 10 classes
  W_fc2 = tf.Variable(tf.truncated_normal(shape=[384, 10], stddev=5e-2))
  b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))
  logits = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
  y_pred = tf.nn.softmax(logits)

  return y_pred, logits

In [0]:
x = tf.placeholder(tf.float32, shape=[None,32,32,3])
y = tf.placeholder(tf.float32, shape=[None,10])
keep_prob = tf.placeholder(tf.float32)

# Parameter
learningRate = 1e-3
epochs = 10000
batchSize = 128
displayStep = 100
testCount = 10

#scalar -> one-hot encoding타입으로 변환
y_train_one_hot = tf.squeeze(tf.one_hot(y_train, 10), axis=1)
y_test_one_hot  = tf.squeeze(tf.one_hot(y_test,  10), axis=1)

y_pred, logits = build_CNN_classifier(x)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y))
train_step = tf.train.RMSPropOptimizer(learningRate).minimize(loss)

#학습이 끝나면 모델의 정확도를 확인한다.
correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [0]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())

  # epochs만큼 최적화를 수행한다.
  for i in range(epochs):
    batch = next_batch(batchSize, x_train, y_train_one_hot.eval())

    # Training이 진행되는 중 train의 상태를 확인하기 위해서 Validation을 하는 코드
    # 그러므로 keep_prob: 1.0으로 설정해야함.
    if i % displayStep==0:
      train_accuracy = accuracy.eval(feed_dict={x:batch[0], y:batch[1], keep_prob:1.0})
      loss_print = loss.eval(feed_dict={x:batch[0], y:batch[1], keep_prob:1.0})
      print("Epoch:%d, acuuracy:%f, loss:%f" %(i,train_accuracy, loss_print))
    
    #Train을 하는 코드
    sess.run(train_step, feed_dict={x:batch[0], y:batch[1], keep_prob:0.8})

  #Training이 완료되면 테스트 데이터셋으로 정확도를 측정한다.
  test_accuracy=0.0
  batchSize = len(x_test) / testCount
  for i in range(testCount):
    test_batch = next_batch(batchSize, x_test, y_test_one_hot.eval())
    test_accuracy = test_accuracy + accuracy.eval(feed_dict={x:test_batch[0], y:test_batch[1], keep_prob:1.0})
  test_accuracy = test_accuracy / testCount
  print("test_accuracy: %f" % test_accuracy)
